In [1]:
day_obs = "2025-07-01"
#day_obs = "yesterday"
show_joined = False

# Identify potential bad images for {{ params.day_obs }}

This notebook uses information from the ConsDb and the EFD to try to identify potential bad visits.

Visits acquired from the FeatureBasedScheduler are tracked in the observatory system through several stages:
* a target event is issued when the visit is requested
* when this requested target is next to be observed, a nextvisit event is issued
* an observation event is issued when the script to acquire the event is registered as successful by the Scheduler
* as the visit is processed and metadata lands in the ConsDb, visit information including zeropoints measured by Rapid Analysis (RA) become available

These can be linked: target + observation are linked by `target_id`, target + nextvisit are linked by the `script sal index`, and nextvisit + consdb visit can be linked by `group_id`.

If the target is requested and visits are acquired, but the observation is never registered, then the script may have been interrupted  by a fault which would also cause problems for the visit.

If the zeropoint reported by RA is significantly far from the predicted zeropoint (currently using an ad-hoc per-visit magnitude cutoff), then it is likely that there were problems with the visit.

In each of these cases, we can identify the potentially troublesome visits and then check by eye in RubinTV -- links are provided to the full mosaic and the witness detector (at USDF).

---------

In [2]:
!pip install --user --upgrade git+https://github.com/lsst-sims/rubin_nights.git --no-deps  > /dev/null 2>&1

In [3]:
import rubin_nights
print('rubin_nights version', rubin_nights.__version__)

rubin_nights version 0.1.dev67+g8996b81.d20250702


In [4]:
import os
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
#from astroplan import Observer

from rubin_nights import connections, scriptqueue
import rubin_nights.dayobs_utils as rn_dayobs
from rubin_nights.targets_and_visits import targets_and_visits
    
import matplotlib.pyplot as plt
from IPython.display import display, Markdown, HTML

import logging
logging.getLogger('rubin_nights').setLevel(logging.INFO)

----------

In [5]:
if os.getenv("EXTERNAL_INSTANCE_URL") is None:
    tokenfile = '/Users/lynnej/.lsst/usdf_rsp'
    site = 'usdf'
else:
    tokenfile = None
    site = None
endpoints = connections.get_clients(tokenfile=tokenfile, site=site)

In [6]:
if isinstance(day_obs, str):
    if day_obs.lower() == "today":
        day_obs = rn_dayobs.today_day_obs()
    elif day_obs.lower() == "yesterday":
        day_obs = rn_dayobs.yesterday_day_obs()

try:
    int(day_obs)
    day_obs = rn_dayobs.day_obs_int_to_str(day_obs)
except ValueError:
    pass
    
day_obs_time = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
#observer = Observer.at_site('Rubin')
#sunset = Time(observer.sun_set_time(day_obs_time, which='next', horizon=-10*u.deg), format='jd')
#sunrise = Time(observer.sun_rise_time(day_obs_time, which='next', horizon=-10*u.deg), format='jd')
sunset = Time(f"{day_obs}T12:00:00", format='isot', scale='tai')
sunrise = Time(f"{day_obs}T12:00:00", format='isot', scale='tai') + TimeDelta(1, format='jd')
print(f"Checking visits on {day_obs}, from {sunset.iso} to {sunrise.iso}")
print(f"Time of notebook execution {Time.now().iso}")

endpoints = connections.get_clients(tokenfile, site)
vt, cols, to, nv, visits = targets_and_visits(sunset, sunrise, endpoints)
print(f"Found {len(vt)} targets->visits.")
print(f"Visit metadata included {visits.scheduler_note.unique()}, {visits.target_name.unique()}")

Checking visits on 2025-07-01, from 2025-07-01 12:00:00.000 to 2025-07-02 12:00:00.000
Time of notebook execution 2025-07-02 09:32:49.774
Found 604 targets->visits.
Visit metadata included ['All_day_clear' '' 'closed_loop_hexapods' 'closed_loop_alldofs'
 'Rubin_SV_225_-40' 'blob_long, iz, a' 'blob_long, iz, b' 'templates, z'
 'pair_33, iz, a' 'pair_33, iz, b' 'templates, r' 'templates, i'
 'pair_15, ri, a' 'pair_15, ri, b'], ['AzEl' 'on-sky flat' 'Rubin_SV_225_-40' 'bulgy' 'bulgy, lowdust'
 'lowdust' 'dusty_plane, bulgy' 'dusty_plane' 'dusty_plane, lowdust' 'nes']


In [7]:
uri = "https://raw.githubusercontent.com/lsst-dm/excluded_visits/" "refs/heads/main/LSSTCam/bad.ecsv"
bad_visits = pd.read_csv(uri, comment="#")
bad_visit_list = bad_visits.exposure.to_list()
bad_visit_dayobs  = [v for v in bad_visit_list if str(v)[0:8] == day_obs.replace('-', '')]

In [8]:
# Were there visits where the rotation angle was changed after the request/next visit?
changed_rot = vt[abs(vt.skyAngle - vt.sky_rotation) > 1]
if len(changed_rot) > 0:
    print(f"Visits with mismatch in sky_rotation (acquired rotation) compared to skyAngle in target request (+ nextvisit). - {len(changed_rot)}")
    display(changed_rot[cols])
else:
    print("No visits with mismatch in sky_rotation (acquired rotation) compared to skyAngle in target request (+ nextvisit).")

No visits with mismatch in sky_rotation (acquired rotation) compared to skyAngle in target request (+ nextvisit).


In [9]:
rubintv_base = "https://usdf-rsp.slac.stanford.edu/rubintv/summit-usdf/lsstcam/event?key=lsstcam/"
def rubintv_links(day_obs, seq_num):
    witness_link = f"{rubintv_base}{day_obs}/witness_detector/{seq_num :06d}/lsstcam_witness_detector_{day_obs}_{seq_num :06d}.jpg"
    witness_html = f'<a href="{witness_link}" target="_blank" rel="noreferrer noopener">{witness_link}</a>'
    mosaic_link = f"{rubintv_base}{day_obs}/calexp_mosaic/{seq_num :06d}/lsstcam_calexp_mosaic_{day_obs}_{seq_num :06d}.jpg"
    mosaic_html = f'<a href="{mosaic_link}" target="_blank" rel="noreferrer noopener">{mosaic_link}</a>'
    return mosaic_html, witness_html

In [11]:
pcols = ['visit_id', 'day_obs', 'seq_num', 'time_target', 'time_observation', 'obs_start', 'scriptSalIndex', 
          'target_name', 's_ra', 's_dec', 'sky_rotation', 'band', 'psf_sigma_median',
         'zero_point_1s', 'zero_point_1s_pred', 'clouds', 'science_program', 'target_name']

if len(vt) == 0:
    print("No observations.")

else:
    quicklook_missing = np.where(np.isnan(vt.zero_point_median) & (vt.visit_id > 0))[0]
    big_zp_offset = np.where((vt.zero_point_1s_pred.values - vt.zero_point_1s.values) > 1.5)[0]
    failed_obs = np.where(np.isnan(vt.time_observation.values) & (vt.visit_id > 0))[0]
    issues = np.concatenate([quicklook_missing, big_zp_offset, failed_obs])
    issues = np.sort(issues)
    issues = np.unique(issues)
    print(f"Found {len(issues)} images with potential issues out of {len(vt)} visits {(len(issues)/len(vt))*100 :.1f}%")
    
    print("From the bad visit list: bad visits from the list which are not flagged here: ")
    print(set(bad_visit_dayobs).difference(set(vt.iloc[issues]['visit_id'].values)))
    
    if len(issues) == 0:
        print("No obvious issues found.")
    else:
        display(HTML("<hr>"))
        display(HTML("Potential problems (please only report OBJECT img_types)"))
        display(HTML("<br>"))
        for idx in issues:
            problem_string = f"IMG_TYPE {vt.iloc[idx].img_type}. Flags:"
            if idx in failed_obs:
                problem_string += " no observation event,"
            if idx in quicklook_missing:
                problem_string += " no quicklook,"
            if idx in big_zp_offset:
                problem_string += " big zeropoint offset,"
            problem_string = problem_string[0:-1] + '.'
            if vt.iloc[idx].visit_id in bad_visit_list:
                problem_string += " Already in bad visit list."
            #problem_string = ", ".join(problem_string)
            display(pd.DataFrame(vt.iloc[idx][pcols]).T)
            print(problem_string)
            mosaic_html, witness_html = rubintv_links(day_obs, vt.iloc[idx].seq_num)
            display(HTML(mosaic_html))
            display(HTML(witness_html))
            display(HTML("<br>"))
        display(HTML("<hr>"))

Found 119 images with potential issues out of 604 visits 19.7%
From the bad visit list: bad visits from the list which are not flagged here: 
set()


,visit_id,day_obs,seq_num,time_target,time_observation,obs_start,scriptSalIndex,target_name,s_ra,s_dec,sky_rotation,band,psf_sigma_median,zero_point_1s,zero_point_1s_pred,clouds,science_program,target_name
53,2025070100176,20250701,176,2025-07-02 00:18:00.359432+00:00,2025-07-02 00:22:18.957480+00:00,2025-07-02T00:22:24.909000,110232,bulgy,269.954634,-32.997725,128.011129,i,2.851961,NaN,28.228636,NaN,BLOCK-365,bulgy


IMG_TYPE science. Flags: no quicklook.


,visit_id,day_obs,seq_num,time_target,time_observation,obs_start,scriptSalIndex,target_name,s_ra,s_dec,sky_rotation,band,psf_sigma_median,zero_point_1s,zero_point_1s_pred,clouds,science_program,target_name
116,2025070100238,20250701,238,2025-07-02 01:12:27.015357+00:00,2025-07-02 01:13:43.289890+00:00,2025-07-02T01:13:48.928000,110295,bulgy,281.806677,-22.238534,117.308075,z,2.561881,26.346785,27.847514,-1.500729,BLOCK-365,bulgy


IMG_TYPE science. Flags: big zeropoint offset.


...

IMG_TYPE science. Flags: big zeropoint offset.


,visit_id,day_obs,seq_num,time_target,time_observation,obs_start,scriptSalIndex,target_name,s_ra,s_dec,sky_rotation,band,psf_sigma_median,zero_point_1s,zero_point_1s_pred,clouds,science_program,target_name
600,2025070100751,20250701,751,2025-07-02 08:28:26.819151+00:00,2025-07-02 08:29:49.262219+00:00,2025-07-02T08:29:54.402000,110781,lowdust,340.819837,-15.115489,88.170657,r,3.007138,24.686067,28.464647,-3.77858,BLOCK-365,lowdust


IMG_TYPE science. Flags: big zeropoint offset.


,visit_id,day_obs,seq_num,time_target,time_observation,obs_start,scriptSalIndex,target_name,s_ra,s_dec,sky_rotation,band,psf_sigma_median,zero_point_1s,zero_point_1s_pred,clouds,science_program,target_name
601,2025070100752,20250701,752,2025-07-02 08:29:10.172106+00:00,2025-07-02 08:30:33.426531+00:00,2025-07-02T08:30:38.918000,110782,lowdust,343.695727,-14.934235,92.743198,r,2.917412,NaN,28.464057,NaN,BLOCK-365,lowdust


IMG_TYPE science. Flags: no quicklook.


In [13]:
if True: #len(failed_obs) > 0:
    efd_and_messages, ecols = scriptqueue.get_consolidated_messages(sunset, sunrise, endpoints)
    mtimes = pd.to_datetime(efd_and_messages.time)
    for idx in failed_obs:
        display(HTML("<hr>"))
        display(HTML("ScriptQueue info on visits missing observations (cause of failure?)"))
        display(HTML("<br>"))
        display(pd.DataFrame(vt.iloc[idx][pcols]).T)
        display(HTML("ScriptQueue Information with same scriptSalIndex"))
        fo = vt.iloc[idx]
        fo_time = pd.to_datetime(fo.time_target)
        mm = efd_and_messages.iloc[np.where(abs(mtimes - fo_time) < pd.Timedelta(1, unit='hr'))[0]]
        mm = mm.query('script_salIndex == @fo.scriptSalIndex')        
        display(HTML(mm[ecols].to_html()))
        display(Markdown("--"))

INFO:rubin_nights.scriptqueue:No queue ENABLED events during time interval 2025-07-01T12:00:00.000 to 2025-07-02T12:00:00.000 for any queue.
INFO:rubin_nights.scriptqueue:Found 1166 script status messages
INFO:rubin_nights.scriptqueue:Found 20 scheduler configuration records
INFO:rubin_nights.scriptqueue:Found 26 messages in the narrative log
INFO:rubin_nights.scriptqueue:Found 57 error messages
INFO:rubin_nights.scriptqueue:Found 700 image times for MTCamera Simonyi
INFO:rubin_nights.scriptqueue:Found 337 image times for ATCamera AuxTel
INFO:rubin_nights.scriptqueue:Found 0 messages in the exposure log


ValueError: Could not convert object to NumPy timedelta

In [36]:
t = endpoints['efd'].select_time_series("lsst.sal.Scheduler.logevent_target", '*', sunset, sunrise)
t.index = t.index - pd.Timedelta(1, 'ms')
t

,airmass,alt,az,blockId,cloud,decl,exposureTimes0,exposureTimes1,exposureTimes2,exposureTimes3,...,skyBrightness,slewTime,snapshotUri,solarElong,sunAlt,sunAz,sunDec,sunRa,targetId,targetName
time,,,,,,,,,,,,,,,,,,,,,
2025-07-01 12:07:58.426043+00:00,1.307902,0,0,303553,0,2.220861,30.0,0.0,0.0,0.0,...,2.000000,4.605422,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,8,SimTarget
2025-07-01 12:17:58.677764+00:00,1.242087,0,0,303555,0,-0.245179,30.0,0.0,0.0,0.0,...,2.000000,4.882803,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,9,SimTarget
2025-07-01 12:27:59.414033+00:00,1.301483,0,0,303557,0,2.425899,30.0,0.0,0.0,0.0,...,2.000000,5.106200,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,10,SimTarget
2025-07-01 12:37:59.461362+00:00,1.240349,0,0,303559,0,-0.038808,30.0,0.0,0.0,0.0,...,2.000000,4.889016,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,11,SimTarget
2025-07-01 12:48:00.152097+00:00,1.295320,0,0,303561,0,2.621555,30.0,0.0,0.0,0.0,...,2.000000,5.122360,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,12,SimTarget
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-02 08:28:26.819151+00:00,1.057579,0,0,110781,0,-15.115448,30.0,0.0,0.0,0.0,...,21.128848,120.000000,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,1859,lowdust
2025-07-02 08:29:10.172106+00:00,1.082980,0,0,110782,0,-14.934172,30.0,0.0,0.0,0.0,...,21.106389,120.000000,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,1860,lowdust
2025-07-02 08:29:50.336656+00:00,1.098863,0,0,110783,0,-14.724051,30.0,0.0,0.0,0.0,...,21.094823,120.000000,https://s3.cp.lsst.org/rubinobs-lfa-cp/Schedul...,0,0,0,0,0,1861,lowdust


In [ ]:
# Targets which did not result in a visit
cols = ['visit_id', 'day_obs', 'seq_num', 'time_target', 'time_observation', 'time_nextvisit', 'groupId', 
        'ra', 'decl', 'skyAngle', 'filter', 'scriptSalIndex', 'note']
if len(vt) > 0:
    incomplete_target = vt.query('visit_id == 0')[cols]
    if len(incomplete_target) == 0:
        print("All requested targets resulted in visits.")
    else:
        print("Targets requested by not observed.")
        display(HTML(incomplete_target.to_html()))